In [4]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:

from transformers import pipeline, set_seed

import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

2024-04-23 12:42:44.849715: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 12:42:44.849857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 12:42:45.035996: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:

import torch

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

loading model and do testing

In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_ckpt = "nijpadariya/Pegasus_fine_tune"
# model_ckpt2 = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained("nijpadariya/Pegasus_fine_tune")
# tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [6]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e238ed61374e12058feacd4aa3d92db0de80fba8e76e43ff209408ebbe474f3a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [8]:
from datasets import load_metric
from tqdm import tqdm

rouge_metric = load_metric("rouge")
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

/tmp/ipykernel_34/257967448.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")
/opt/conda/lib/python3.10/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [9]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [10]:
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score

In [7]:
model_pegasus

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

In [8]:
tokenizer

PegasusTokenizerFast(name_or_path='/kaggle/input/tokenizer', vocab_size=96103, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>', 

In [3]:
pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


Here we first Original CNN3000Hindi Dataset translate to english(using IndicTrans) and Load that translated file here and that will pass through fine_Tuned Pegasus model 

In [2]:
import pandas as pd

# Specify the path to the CSV file
input_csv_path = '/kaggle/input/data-set/CNN_3000_translated_english_IndicTrans_main.csv'

# Read the CSV file into a DataFrame
df_with_summaries = pd.read_csv(input_csv_path)

# Display the DataFrame (optional)
# print(df_with_summaries.head())  # Display the first few rows of the DataFrame

In [4]:
df_with_summaries['article_english'][1]

"Child killer Ronald Salazar, who has served a decade of a life sentence for the gruesome murder of his sister, returns to court after the U.S. Supreme Court ruled in 2012 to ban mandatory life sentences for juvenile murderers without the possibility of parole. Salazar asks Miami-Dade Street Jail Judge Ellen Sue Wenger to re-sentence him. Salazar asks Salazar to re-sentence a juvenile murderer who has served a decade of a life sentence for the gruesome murder of his sister. Salazar apologizes after re-sentencing. Salazar admits that Salazar tried again. Salazar admits that his whole life was' messed up 'after Salazar. Salazar tries again. Salazar admits that Salazar tried to rape his mother-in-law after 11 years in prison."

In [9]:
inputs = tokenizer(df_with_summaries['article_english'][0], return_tensors="pt", max_length=1024, truncation=True)
inputs = {key: value.to(model_pegasus.device) for key, value in inputs.items()}
# Generate summary
summary_ids = model_pegasus.generate(inputs["input_ids"], max_length=200, num_beams=8, length_penalty=0.0, early_stopping=True)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary)

Alex Song has completed a season-long loan to West Ham from Arsenal.<n>The midfielder posted a picture of his outfit on Instagram on his way to training.


In [13]:
model_summary=[]
for i in range(3000):
    inputs = tokenizer(df_with_summaries['article_english'][i], return_tensors="pt", max_length=1024, truncation=True)
    inputs = {key: value.to(model_pegasus.device) for key, value in inputs.items()}
    # Generate summary
    summary_ids = model_pegasus.generate(inputs["input_ids"], max_length=150, num_beams=8, length_penalty=0.0, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    model_summary.append(summary)
#     print(inputs)
#     print(summary_ids)
#     print("Input Text:", en_paragraph)
#     print("Summary:", summary)


In [ ]:
df_with_summaries['hightlight_english_pred']=''
for i in range(3000):
    df_with_summaries['hightlight_english_pred'][i]=model_summary[i]
    

In [21]:
model_summary[497]

'Britons have donated £55 million to a small village on the island of Calagnaan in the Philippines. Typhoon survivors have helped more than half a million people affected by the typhoon.'

In [22]:
# import pandas as pd
# Convert translated_data to DataFrame
df_translated = pd.DataFrame(df_with_summaries)

# Save the translated DataFrame to a CSV file
df_translated.to_csv('CNN_3000_translated_English_summarized_English_main.csv', index=False)

Now CNN_3000_translated_English_summarized_English_main.csv file pass through IndicTrans again and translate to Hindi again then load that translated file here to calculate Rouge Score and Similiarity Score

In [1]:
import pandas as pd

# Specify the path to the CSV file
input_csv_path = '/kaggle/input/dataset/CNN_3000_translated_English_to_Hindi_Again__summarized.csv'

# Read the CSV file into a DataFrame
df_with_summaries = pd.read_csv(input_csv_path)

In [ ]:
from sentence_transformers import SentenceTransformer, util
# sentences = ["It is a guide to action which ensures that the military always obeys the commands of the party", "It is a guide to action that ensures that the military will forever heed Party commands"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

final_val=0
count=0
#Compute embedding for both lists
for i in range(3000):
    embedding_1= model.encode(df_with_summaries['highlights_english'][i], convert_to_tensor=True)
    embedding_2 = model.encode(df_with_summaries['hightlight_english_pred'][i], convert_to_tensor=True)

    val=util.pytorch_cos_sim(embedding_1, embedding_2)
    final_val=final_val+val.item()
    count=count+1
## tensor([[0.6003]])

In [8]:
final_val=final_val/count
print(final_val)

0.6197199414601785


In [12]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
# Calculate ROUGE scores for each pair of summaries
from rouge import Rouge
rouge = Rouge()
scores_list = []
for idx, (SystemSummary1, ground) in enumerate(zip(df_with_summaries['hightlight_english_pred'], df_with_summaries['highlights_english'])):
    try:
        # Check if either SystemSummary1 or ground is empty
        if not SystemSummary1.strip() or not ground.strip():
            print(f"Skipping empty pair of summaries in row {idx}.")
            continue

        # Convert SystemSummary1 and ground to strings
        SystemSummary1 = str(SystemSummary1)
        ground = str(ground)

        # Calculate ROUGE scores
        scores = rouge.get_scores(SystemSummary1, ground)
        scores_list.append(scores[0])

    except ValueError as e:
        print(f"Skipping row {idx} due to error: {e}")



In [14]:
# Calculate the average ROUGE scores for each metric across all pairs
average_scores = {}
for scores in scores_list:
    for metric, values in scores.items():
        for key, value in values.items():
            column_name = f"{metric}_{key}"
            if column_name not in average_scores:
                average_scores[column_name] = []
            average_scores[column_name].append(value)

# Calculate the overall average scores for each metric
overall_scores = {}
for metric, values in average_scores.items():
    overall_scores[metric] = sum(values) / len(values)

# Create a DataFrame to display the overall scores
overall_scores_df = pd.DataFrame(overall_scores, index=['average'])
print(overall_scores_df)


         rouge-1_r  rouge-1_p  rouge-1_f  rouge-2_r  rouge-2_p  rouge-2_f  \
average   0.234996   0.401981     0.2869   0.085709   0.152956   0.105887   

         rouge-l_r  rouge-l_p  rouge-l_f  
average   0.215126   0.369035   0.262987  
